# TEMPO 데이터 다운로드

NASA Earthdata에서 TEMPO (Tropospheric Emissions: Monitoring of Pollution) 데이터를 다운로드하는 스크립트입니다.

## 다운로드 대상 데이터
- **NO₂ (이산화질소)**: V03 + V04 (Standard), V02 NRT (Near Real-Time)
- **HCHO (포름알데히드)**: V04 (Standard), V02 NRT
- **O₃ (오존)**: V04 (Standard)
- **Cloud (구름)**: V04 (Standard), V02 NRT

## 1. 설치 및 Import

In [ ]:
# earthaccess 라이브러리 설치 (처음 실행 시)
# !pip install earthaccess pandas

In [1]:
import earthaccess
import pandas as pd
from datetime import datetime
from pathlib import Path

c:\Users\YJKIM_PC\Documents\나사 해커톤 2025\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. NASA Earthdata 인증

처음 실행 시 NASA Earthdata 계정 정보를 입력해야 합니다.
- 계정이 없다면: https://urs.earthdata.nasa.gov/ 에서 가입

In [2]:
# NASA Earthdata 로그인
auth = earthaccess.login()

if auth:
    print("✅ 인증 성공!")
else:
    print("❌ 인증 실패. 다시 시도해주세요.")

✅ 인증 성공!


## 3. 다운로드 설정

In [3]:
# 다운로드할 데이터셋 정의 (CSV 파일 기반)
datasets = {
    'NO2': [
        {'short_name': 'TEMPO_NO2_L2', 'version': 'V03'},
        {'short_name': 'TEMPO_NO2_L3', 'version': 'V03'},
        {'short_name': 'TEMPO_NO2_L2', 'version': 'V04'},
        {'short_name': 'TEMPO_NO2_L3', 'version': 'V04'},
        {'short_name': 'TEMPO_NO2_L2_NRT', 'version': 'V02'},
        {'short_name': 'TEMPO_NO2_L3_NRT', 'version': 'V02'},
    ],
    'HCHO': [
        {'short_name': 'TEMPO_HCHO_L2', 'version': 'V03'},
        {'short_name': 'TEMPO_HCHO_L3', 'version': 'V03'},
        {'short_name': 'TEMPO_HCHO_L2', 'version': 'V04'},
        {'short_name': 'TEMPO_HCHO_L3', 'version': 'V04'},
        {'short_name': 'TEMPO_HCHO_L2_NRT', 'version': 'V02'},
        {'short_name': 'TEMPO_HCHO_L3_NRT', 'version': 'V02'},
    ],
    'O3': [
        {'short_name': 'TEMPO_O3TOT_L2', 'version': 'V03'},
        {'short_name': 'TEMPO_O3TOT_L3', 'version': 'V03'},
        {'short_name': 'TEMPO_O3TOT_L2', 'version': 'V04'},
        {'short_name': 'TEMPO_O3TOT_L3', 'version': 'V04'},
        # 선택사항: Profile 데이터
        # {'short_name': 'TEMPO_O3PROF_L2', 'version': 'V04'},
        # {'short_name': 'TEMPO_O3PROF_L3', 'version': 'V04'},
    ],
    'Cloud': [
        {'short_name': 'TEMPO_CLDO4_L2', 'version': 'V03'},
        {'short_name': 'TEMPO_CLDO4_L3', 'version': 'V03'},
        {'short_name': 'TEMPO_CLDO4_L2', 'version': 'V04'},
        {'short_name': 'TEMPO_CLDO4_L3', 'version': 'V04'},
        {'short_name': 'TEMPO_CLDO4_L2_NRT', 'version': 'V02'},
        {'short_name': 'TEMPO_CLDO4_L3_NRT', 'version': 'V02'},
    ]
}

In [4]:
# 다운로드 설정
config = {
    # 시간 범위 (TEMPO 전체 데이터: 2023년 8월 ~ 2024년 12월, 약 17개월)
    'temporal': ('2023-08-01', '2024-12-31'),
    
    # 공간 범위 (bounding box: min_lon, min_lat, max_lon, max_lat)
    # 예: 캘리포니아 주 (California)
    'bounding_box': (-124.4, 32.5, -114.1, 42.0),
    
    # 또는 다른 주 선택:
    # Texas: (-106.6, 25.8, -93.5, 36.5)
    # New York: (-79.8, 40.5, -71.9, 45.0)
    # Florida: (-87.6, 24.5, -80.0, 31.0)
    
    # 다운로드 폴더
    'download_dir': './tempo_data',
    
    # 파일 개수 제한 (테스트용, None = 전체 다운로드)
    'max_files_per_dataset': 10,  # 테스트: 각 데이터셋당 10개 파일만
    # 'max_files_per_dataset': None,  # 실제 사용: 전체 다운로드
}

# 다운로드 폴더 생성
Path(config['download_dir']).mkdir(parents=True, exist_ok=True)

print(f"⚙️ 다운로드 설정:")
print(f"  - 기간: {config['temporal'][0]} ~ {config['temporal'][1]}")
print(f"  - 지역: {config['bounding_box']}")
print(f"  - 저장 경로: {config['download_dir']}")
print(f"  - 파일 제한: {config['max_files_per_dataset']} (per dataset)")

⚙️ 다운로드 설정:
  - 기간: 2023-08-01 ~ 2024-12-31
  - 지역: (-124.4, 32.5, -114.1, 42.0)
  - 저장 경로: ./tempo_data
  - 파일 제한: 10 (per dataset)


## 4. 데이터 검색 및 다운로드

In [6]:
def search_and_download_tempo_data(dataset_type, dataset_info, config):
    """
    TEMPO 데이터를 검색하고 다운로드합니다.
    
    Args:
        dataset_type: 데이터 타입 (NO2, HCHO, O3, Cloud)
        dataset_info: {'short_name': str, 'version': str}
        config: 다운로드 설정 딕셔너리
    """
    short_name = dataset_info['short_name']
    version = dataset_info['version']
    
    print(f"\n{'='*60}")
    print(f"🔍 검색 중: {dataset_type} - {short_name} ({version})")
    print(f"{'='*60}")
    
    try:
        # 데이터 검색
        results = earthaccess.search_data(
            short_name=short_name,
            version=version,
            temporal=config['temporal'],
            bounding_box=config['bounding_box'],
        )
        
        if not results:
            print(f"⚠️  검색 결과 없음")
            return []
        
        # 파일 개수 제한
        if config['max_files_per_dataset']:
            results = results[:config['max_files_per_dataset']]
        
        print(f"📊 검색 결과: {len(results)}개 파일")
        
        # 다운로드
        print(f"⬇️  다운로드 시작...")
        download_path = Path(config['download_dir']) / dataset_type / short_name / version
        download_path.mkdir(parents=True, exist_ok=True)
        
        downloaded_files = earthaccess.download(
            results,
            str(download_path)
        )
        
        print(f"✅ 다운로드 완료: {len(downloaded_files)}개 파일")
        print(f"   저장 경로: {download_path}")
        
        return downloaded_files
        
    except Exception as e:
        print(f"❌ 오류 발생: {str(e)}")
        return []

In [ ]:
# 모든 데이터셋 다운로드
download_summary = []

for dataset_type, dataset_list in datasets.items():
    print(f"\n{'#'*70}")
    print(f"#  {dataset_type} 데이터 다운로드 시작")
    print(f"{'#'*70}")
    
    for dataset_info in dataset_list:
        files = search_and_download_tempo_data(dataset_type, dataset_info, config)
        
        download_summary.append({
            'Type': dataset_type,
            'Short Name': dataset_info['short_name'],
            'Version': dataset_info['version'],
            'Files Downloaded': len(files)
        })

print(f"\n\n{'='*70}")
print("🎉 모든 다운로드 완료!")
print(f"{'='*70}")


######################################################################
#  NO2 데이터 다운로드 시작
######################################################################

🔍 검색 중: NO2 - TEMPO_NO2_L2 (V03)
📊 검색 결과: 10개 파일
⬇️  다운로드 시작...


QUEUEING TASKS | : 100%|██████████| 10/10 [00:00<00:00, 666.64it/s]
PROCESSING TASKS | : 100%|██████████| 10/10 [01:32<00:00,  9.29s/it]
COLLECTING RESULTS | : 100%|██████████| 10/10 [00:00<?, ?it/s]


✅ 다운로드 완료: 10개 파일
   저장 경로: tempo_data\NO2\TEMPO_NO2_L2\V03

🔍 검색 중: NO2 - TEMPO_NO2_L3 (V03)
📊 검색 결과: 10개 파일
⬇️  다운로드 시작...


QUEUEING TASKS | : 100%|██████████| 10/10 [00:00<00:00, 714.24it/s]
PROCESSING TASKS | : 100%|██████████| 10/10 [13:09<00:00, 78.98s/it]
COLLECTING RESULTS | : 100%|██████████| 10/10 [00:00<?, ?it/s]


✅ 다운로드 완료: 10개 파일
   저장 경로: tempo_data\NO2\TEMPO_NO2_L3\V03

🔍 검색 중: NO2 - TEMPO_NO2_L2 (V04)
⚠️  검색 결과 없음

🔍 검색 중: NO2 - TEMPO_NO2_L3 (V04)
⚠️  검색 결과 없음

🔍 검색 중: NO2 - TEMPO_NO2_L2_NRT (V02)
⚠️  검색 결과 없음

🔍 검색 중: NO2 - TEMPO_NO2_L3_NRT (V02)
⚠️  검색 결과 없음

######################################################################
#  HCHO 데이터 다운로드 시작
######################################################################

🔍 검색 중: HCHO - TEMPO_HCHO_L2 (V03)
📊 검색 결과: 10개 파일
⬇️  다운로드 시작...


...

QUEUEING TASKS | : 100%|██████████| 10/10 [00:00<00:00, 1165.70it/s]
PROCESSING TASKS | : 100%|██████████| 10/10 [03:52<00:00, 23.21s/it]
COLLECTING RESULTS | : 100%|██████████| 10/10 [00:00<?, ?it/s]


✅ 다운로드 완료: 10개 파일
   저장 경로: tempo_data\O3\TEMPO_O3TOT_L3\V03

🔍 검색 중: O3 - TEMPO_O3TOT_L2 (V04)
⚠️  검색 결과 없음

🔍 검색 중: O3 - TEMPO_O3TOT_L3 (V04)
⚠️  검색 결과 없음

######################################################################
#  Cloud 데이터 다운로드 시작
######################################################################

🔍 검색 중: Cloud - TEMPO_CLDO4_L2 (V03)
📊 검색 결과: 10개 파일
⬇️  다운로드 시작...


QUEUEING TASKS | : 100%|██████████| 10/10 [00:00<00:00, 499.87it/s]
PROCESSING TASKS | : 100%|██████████| 10/10 [00:31<00:00,  3.15s/it]
COLLECTING RESULTS | : 100%|██████████| 10/10 [00:00<?, ?it/s]


✅ 다운로드 완료: 10개 파일
   저장 경로: tempo_data\Cloud\TEMPO_CLDO4_L2\V03

🔍 검색 중: Cloud - TEMPO_CLDO4_L3 (V03)
📊 검색 결과: 10개 파일
⬇️  다운로드 시작...


QUEUEING TASKS | : 100%|██████████| 10/10 [00:00<00:00, 454.41it/s]
PROCESSING TASKS | :  50%|█████     | 5/10 [01:55<01:03, 12.70s/it]

## 5. 다운로드 결과 확인

In [ ]:
# 다운로드 요약 테이블
summary_df = pd.DataFrame(download_summary)
print("\n📋 다운로드 요약:")
print(summary_df.to_string(index=False))

total_files = summary_df['Files Downloaded'].sum()
print(f"\n📦 총 다운로드 파일 수: {total_files}개")

In [ ]:
# 다운로드 결과를 CSV로 저장
summary_filename = f"download_summary_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
summary_df.to_csv(summary_filename, index=False)
print(f"\n💾 다운로드 요약 저장: {summary_filename}")

## 6. 다운로드된 파일 구조 확인

In [ ]:
# 다운로드 폴더 구조 확인
import os

def print_directory_tree(path, prefix="", max_depth=3, current_depth=0):
    """디렉토리 구조를 트리 형태로 출력"""
    if current_depth >= max_depth:
        return
    
    path = Path(path)
    if not path.exists():
        return
    
    items = list(path.iterdir())
    items.sort()
    
    for i, item in enumerate(items):
        is_last = i == len(items) - 1
        current_prefix = "└── " if is_last else "├── "
        print(f"{prefix}{current_prefix}{item.name}")
        
        if item.is_dir():
            extension_prefix = "    " if is_last else "│   "
            print_directory_tree(item, prefix + extension_prefix, max_depth, current_depth + 1)

print("\n📁 다운로드 폴더 구조:")
print(config['download_dir'])
print_directory_tree(config['download_dir'])

## 추가 참고사항

### 데이터 파일 형식
- TEMPO 데이터는 주로 **netCDF4 (.nc)** 형식으로 제공됩니다
- 파일을 읽으려면 `xarray` 또는 `netCDF4` 라이브러리를 사용하세요

### 다음 단계
1. **데이터 로드 및 전처리**: `xarray.open_dataset()` 사용
2. **시각화**: matplotlib, cartopy 등으로 지도 위에 표시
3. **머신러닝 모델**: 시계열 데이터로 변환 후 학습

### 유용한 링크
- TEMPO 공식 문서: https://tempo.si.edu/
- earthaccess 문서: https://earthaccess.readthedocs.io/
- NASA Earthdata: https://www.earthdata.nasa.gov/